<a href="https://colab.research.google.com/github/VivianPita/lab_iagi/blob/main/Es_7_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Field test del MultiLayer Perceptron

Questo è il testo dell'esercitazione che dovrete completare per questa settimana. Non sarà guidato passo passo, ma
dovrete usare quello che avete imparato negli altri jupyter notebooks per addestrare dei modelli e cross-validarli,
al massimo delle vostre capacità.
I tre esercizi da svolgere sono:

1. addestrare un MLP su [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist). Potete utilizzare il codice
 del precedente notebook, ma vi consiglio di riscriverlo in maniera più ordinata. In questo caso, fate prima
 un'analisi dei dati per capire con cosa avete a che fare.
2. addestrare un MLP su [YearPredictionMSD](https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD): l'idea di
      questo esercizio è quella di utilizzare una rete neurale per migliorare le performance rispetto alla soluzione
      che avete già implementato con la logistic regression nell'[esercitazione 6](https://github.com/Sapienza-AI-Lab/esercitazione6-22-23). In questo caso l'aspetto che ci
      preme esplorare è l'efficacia del MLP nell'imparare una rappresentazione (i.e. delle features) migliore per la
      classifcazione, o la regressione. Questo esercizion non è banale e sono interessato a vedere come sfrutterete
      le reti neurali per migliorare le prestazioni. Se non avete fatto l'EDA durante la scorsa esercitazione, questo è il momento di farla.
3. addestrare un MLP su CIFAR-10: riprenderemo questo dataset anche con le reti convoluzionali, ma iniziamo a
      farci un'idea delle sue caratterisitche addestrando im modello migliore possibile utilizzando un MLP. La sfida
      in questo caso sarà la dimensionalità dell'input, e quindi delle connessioni del MLP.


In tutti i casi tenete conto di questi aspetti:
* Non siete costretti ad usare i notebook. Personalmente non li amo, ma sono adatti a presentare il codice per la
    didattica. Per lavorare con progetti di una certa complessità è spesso meglio passare ad un normale progetto
    python. Ovviamente se non avete una GPU nel vostro portatile/desktop, l'unica soluzione è colab.
* Utilizzate tensorboard o weight and biases per la visualizzazione e per fare il debug del vostro modello. Non provate le cose a caso.


###LIBRERIE + MODEL SET UP


In [1]:
#importo le varie librerie
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, SubsetRandomSampler, ConcatDataset
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import sklearn
import time
from sklearn.model_selection import train_test_split

In [2]:
class MLP(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(MLP,self).__init__()
    self.fc1=nn.Linear(input_size,hidden_size)
    self.fc2=nn.Linear(hidden_size,output_size)
  def forward(self,x):
    x=self.fc1(x)
    x=F.relu(x)
    x=self.fc2(x)
    return x

In [3]:
def train(model, device, optimizer, loss_fn, epochs, train_loader, validation_loader):
    model.train()
    train_losses = []
    validation_losses = []

    for epoch in range(epochs):
        total_loss = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.item()
                ))
        train_losses.append(total_loss / len(train_loader))

        total_loss = 0
        for data, target in validation_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = model(data) #validation using the model with the updated weights and biases from the training
            loss = loss_fn(output, target)
            total_loss += loss.item()
        validation_losses.append(total_loss / len(validation_loader))

    return train_losses, validation_losses

In [4]:
def test(model, device, loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad(): #no need to calculate the gradients when testing
        for data, target in loader: #loader is the test_loader
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = model(data) #do not calculate the gradients because of the line with torch.no_grad()
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True) #keep dim for example is [[1], [2]] instead of [1, 2]
            correct += pred.eq(target.view_as(pred)).sum().item() #sum up the correct predictions
    test_loss /= len(loader.dataset)
    print('Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss,
        correct,
        len(loader.dataset),
        100. * correct / len(loader.dataset)
    ))

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

###FASHIONMNIST